In [41]:
import io
import cv2
import numpy as np
import pickle
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d, Axes3D

In [55]:
pcds = np.load('/root/real2sim/real2sim/real_points/red/real_pcds.npy', allow_pickle=True)
with open('/root/ExPCP/policy/pbm/experts/Move_1000_8000_1000_8000_500_2000/Move-v1/expert_0.0000_13:01:00.pickle', 'rb') as f:
    pcds_sim = pickle.load(f)['plasticine_pc']

In [ ]:
end_index = 150
step = 0

output_pcds = []
for real, sim in zip(pcds[:end_index], pcds_sim[:end_index]):
    real = real[:, [0, 2, 1]]
    # size modify
    real = (real - np.mean(real, axis=0)) * 2 + np.mean(real, axis=0)
    # x modify
    real[:, 0] = 2 - real[:, 0]
    if step == 0:
        real_x_min = np.max(real[:, 0])
        sim_x_min = np.max(sim[:, 0])
        x_diff = real_x_min - sim_x_min
        real[:, 0] -= x_diff
    else:
        real[:, 0] -= x_diff
    # y modify
    if step == 0:
        real_y_min = np.min(real[:, 1])
        real[:, 1] -= real_y_min
    else:
        real[:, 1] -= real_y_min
    # z modify
    real[:, 2] += 0.5
    # cut
    real = real[real[:, 0] > 0.2]
    real = real[real[:, 0] < 0.8]
    real = real[real[:, 2] > 0.4]
    real = real[real[:, 2] < 0.6]
    output_pcds.append(real)
    
    fig = plt.figure()
    ax1 = fig.add_subplot(1, 1, 1, projection='3d')
    ax1.scatter(real[:, 0], real[:, 1], real[:, 2], s=1)
    ax1.scatter(sim[:, 0], sim[:, 1], sim[:, 2], s=1)
    ax1.view_init(90, 270)

    ax1.set_xlim(0, 1)
    ax1.set_ylim(0, 1)
    ax1.set_zlim(0.4, 0.6)
    ax1.set_xlabel('x')
    ax1.set_ylabel('y')
    ax1.set_zlabel('z')